In [1]:
# Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Data
colnames = ['ProductID','Product_type','Date','Price','Quantity']
df = pd.read_csv('beer.csv', names=colnames, header=None)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ProductID     12 non-null     object 
 1   Product_type  12 non-null     object 
 2   Date          12 non-null     object 
 3   Price         12 non-null     float64
 4   Quantity      12 non-null     int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 608.0+ bytes


In [4]:
df.head(10)


,ProductID,Product_type,Date,Price,Quantity
0,S001,Beer,2021-03-30,24.49,21819
1,J001,Beer,2021-03-30,22.99,19836
2,L001,Beer,2021-03-30,21.95,21978
3,S001,Beer,2021-06-30,24.49,19134
4,J001,Beer,2021-06-30,22.99,17690
5,L001,Beer,2021-06-30,21.95,17007
6,S001,Beer,2021-09-30,24.49,19134
7,J001,Beer,2021-09-30,22.99,17690
8,L001,Beer,2021-09-30,21.95,17007
9,S001,Beer,2021-12-30,24.49,19134


In [5]:
#Format and build a dataframe with x_values for each product within the caregory
x_pivot = df.pivot(index= 'Date' ,columns='ProductID' ,values='Price')
x_values = pd.DataFrame(x_pivot.to_records())
print(x_values)

         Date   J001   L001   S001
0  2021-03-30  22.99  21.95  24.49
1  2021-06-30  22.99  21.95  24.49
2  2021-09-30  22.99  21.95  24.49
3  2021-12-30  22.99  21.95  24.49


In [6]:
#Format and build a dataframe with y_values for each product within the caregory
y_pivot = df.pivot( index = 'Date',columns='ProductID', values='Quantity')
y_values = pd.DataFrame(y_pivot.to_records())
print(y_values)

         Date   J001   L001   S001
0  2021-03-30  19836  21978  21819
1  2021-06-30  17690  17007  19134
2  2021-09-30  17690  17007  19134
3  2021-12-30  17690  17007  19134


In [9]:
points = []
results_values = {
    "name": [],
    "price_elasticity": [],
    "price_mean": [],
    "quantity_mean": [],
    "intercept": [],
    "t_score":[],
    "slope": [],
    "rsquared": [],
    "coefficient_pvalue" : [],
}
#Append x_values with y_values per same product name
for column in x_values.columns[1:]:
    column_points = []
    for i in range(len(x_values[column])):
        if not np.isnan(x_values[column][i]) and not np.isnan(y_values[column][i]):
            column_points.append((x_values[column][i], y_values[column][i]))
    df = pd.DataFrame(list(column_points), columns= ['x_value', 'y_value'])


    #Linear Regression Model
    import statsmodels.api as sm
    x_value = df['x_value']
    y_value = df['y_value']
    X = sm.add_constant(x_value)
    model = sm.OLS(y_value, X)
    result = model.fit()
    
    
    #(Null Hypothesis test) Coefficient with a p value less than 0.05
    if result.f_pvalue < 0.05:
        
        rsquared = result.rsquared
        coefficient_pvalue = result.f_pvalue
        intercept, slope = result.params
        mean_price = np.mean(x_value)
        mean_quantity = np.mean(y_value)
        tintercept, t_score = result.tvalues
     
        #Price elasticity Formula
        price_elasticity = (slope)*(mean_price/mean_quantity)    
            
        #Append results into dictionary for dataframe
        results_values["name"].append(column)
        results_values["price_elasticity"].append(price_elasticity)
        results_values["price_mean"].append(mean_price)
        results_values["quantity_mean"].append(mean_quantity)
        results_values["intercept"].append(intercept)
        results_values['t_score'].append(t_score)
        results_values["slope"].append(slope)
        results_values["coefficient_pvalue"].append(coefficient_pvalue)
        
final_df = pd.DataFrame.from_dict(results_values)
df_elasticity = []
final_df[['name','price_elasticity','t_score','coefficient_pvalue','slope','price_mean','quantity_mean','intercept','rsquared']]

,name,price_elasticity,t_score,coefficient_pvalue,slope,price_mean,quantity_mean,intercept,rsquared


In [10]:
def divergent_plot(df, values_column, ylabel, xlabel):

    #Divergent plot
    df['ranking'] = df[values_column].rank( ascending = True).astype(int)
    df.sort_values(values_column, ascending =False, inplace = True)
    plt.figure(figsize = (12,5), dpi = 80)
    plt.hlines(y = df['ranking'] , xmin = 0, xmax = df[values_column], alpha = 0.5, linewidth = 3)
    
    #Add elasticity labels
    for x, y, tex in zip(df[values_column], df['ranking'] , df[values_column]):
        plt.text(x, y, round(tex, 2), horizontalalignment='right' if x < 0 else 'left', 
                 verticalalignment='center', fontdict={'color':'red' if x < 0 else 'green', 'size':10})
        
    
    # Axis and title
    plt.gca().set(ylabel= ylabel, xlabel= xlabel)
    plt.yticks(df['ranking'])
    plt.title(values_column , fontdict={'size':13})
    plt.grid(linestyle='--', alpha=0.5)
    plt.show()
            
    
    #Adjust Ranking column and print dataframe
    pd.set_option('display.width', 4000)
    cols = list(df.columns)
    cols = [cols[-1]] + cols[:-1]
    df = df[cols]
    
    df = df.iloc[:,:3]
    df.set_index('ranking', inplace=True)
    display(df)

In [11]:
pe_plot = divergent_plot(df_elasticity, 'price_elasticity', 'Ranking Number', 'Price Elasticity')


TypeError: list indices must be integers or slices, not str